## Module

In [45]:
import cv2
import numpy as np
from datetime import datetime
import os

def generate_line_image_by_data(data, image_path, point_count = 52, image_size=(512, 512), line_color=(0,0,0,255), line_width=1, data_range=(0, 1), image_margin=(0,0)):

    """Generate line image by data

    Args:
        data (array): data array. value ranges from 0 to 1. 
        image_path (string): absolute path to store the generated image.
        point_count (int, optional): point amount of line. will automatically interpolate if this number does not match the length of data array. Defaults to 52.
        image_size (tuple, optional): image size in pixel. Defaults to (512, 512).
        line_color (tuple, optional): RGBA color. Defaults to (0,0,0,255).
        line_width (int, optional): line thickness. Defaults to 1.
        data_range (tuple, optional): can clamp and remap data value to [0, 1]. Defaults to (0,1).
        image_margin (tuple, optional): margin that image has. value is in percentage. . Defaults to (0,0).
    """
    inArray = data
    originalPosArray = [0] * len(inArray) * 3

    pointCount = int(point_count)
    pointArray = [0] * pointCount * 3

    apiRangeMin = data_range[0]
    apiRangeMax = data_range[1]

    rangeXMin = -1 + image_margin[0]
    rangeXMax = 1 - image_margin[0]
    rangeYMin = -1 + image_margin[1]
    rangeYMax = 1 - image_margin[1]

    # determine all position
    for i in range(len(inArray)):
        originalPosArray[i * 3 + 0] = MapRange(i, 0, len(inArray) - 1, rangeXMin, rangeXMax)
        originalPosArray[i * 3 + 1] = MapRange(inArray[i], apiRangeMin, apiRangeMax, rangeYMin, rangeYMax)
        originalPosArray[i * 3 + 2] = 0

    # line point 
    percentage = 0
    for i in range(pointCount):
        percentage = float(i) / float(pointCount - 1)
        pointArray[i * 3 + 0], pointArray[i * 3 + 1], pointArray[i * 3 + 2] = GetInterpolatedPoint(originalPosArray, percentage)

    # create image
    width = int(image_size[0])
    height = int(image_size[1])
    channel = 4
    image = np.zeros((height, width, channel), np.uint8)

    # draw line
    for i in range(pointCount-1):
        x1, y1 = MapRange(pointArray[i * 3 + 0], -1, 1, 0, width),  MapRange(pointArray[i * 3 + 1], -1, 1, 0, height)
        x2, y2 = MapRange(pointArray[(i+1) * 3 + 0], -1, 1, 0, width), MapRange(pointArray[(i+1) * 3 + 1], -1, 1, 0, height)
        print(x1, y1, x2, y2)
        cv2.line(image, (int(x1), int(y1)), (int(x2), int(y2)), line_color, thickness=int(line_width), lineType=cv2.LINE_AA)

    # output image
    cv2.imwrite(image_path, image)


def GetInterpolatedPoint(in_array, per):
    array_length = len(in_array) // 3

    threshold_index = min(max(0, per), 1) * (array_length - 1)
    start_index = int(threshold_index)
    end_index = min(start_index + 1, array_length - 1)
    lerp_in_segment = threshold_index - start_index

    return [
        MapRange(lerp_in_segment, 0, 1, in_array[start_index * 3 + 0], in_array[end_index * 3 + 0]),
        MapRange(lerp_in_segment, 0, 1, in_array[start_index * 3 + 1], in_array[end_index * 3 + 1]),
        MapRange(lerp_in_segment, 0, 1, in_array[start_index * 3 + 2], in_array[end_index * 3 + 2])
    ]


def MapRange(v, o_min, o_max, n_min, n_max, ease=0):
    if v >= max(o_max, o_min):
        v = n_max
    elif v <= min(o_max, o_min):
        v = n_min
    else:
        reverse_input = False
        old_min = min(o_min, o_max)
        old_max = max(o_min, o_max)
        if old_min != o_min:
            reverse_input = True

        reverse_output = False
        new_min = min(n_min, n_max)
        new_max = max(n_min, n_max)
        if new_min != n_min:
            reverse_output = True

        if reverse_input:
            portion = (old_max - v) * (new_max - new_min) / (old_max - old_min)
        else:
            portion = (v - old_min) * (new_max - new_min) / (old_max - old_min)

        if reverse_output:
            r = new_max - portion
        else:
            r = portion + new_min

        if ease == 0:
            v = r
        elif ease == 1:
            x = max(0, min(1, (r - n_min) / (n_max - n_min)))
            v = n_min + x * x * (3 - 2 * x) * (n_max - n_min)  # smoothstep
        elif ease == 2:
            x = max(0, min(1, (r - n_min) / (n_max - n_min)))
            v = n_min + x * x * x * (x * (x * 6 - 15) + 10) * (n_max - n_min)  # smootherstep

    return v

## Test Case

In [ ]:
response = {"rtnCode":0,"msg":"请求成功","data":[{"RYJJ-06-0002":[{"data":"[{\"index\":0,\"value\":-5.1,\"date\":\"2023-07-10\"},{\"index\":1,\"value\":-4.1,\"date\":\"2023-07-11\"},{\"index\":2,\"value\":5.9,\"date\":\"2023-07-12\"},{\"index\":3,\"value\":-3.1,\"date\":\"2023-07-12\"},{\"index\":4,\"value\":-2.1,\"date\":\"2023-07-13\"},{\"index\":5,\"value\":-1.1,\"date\":\"2023-07-14\"},{\"index\":6,\"value\":-0.1,\"date\":\"2023-07-15\"},{\"index\":7,\"value\":0.9,\"date\":\"2023-07-16\"},{\"index\":8,\"value\":1.9,\"date\":\"2023-07-17\"},{\"index\":9,\"value\":2.9,\"date\":\"2023-07-18\"},{\"index\":10,\"value\":3.9,\"date\":\"2023-07-19\"},{\"index\":11,\"value\":4.9,\"date\":\"2023-07-20\"},{\"index\":12,\"value\":6.9,\"date\":\"2023-07-22\"},{\"index\":13,\"value\":3.6,\"date\":\"2023-07-23\"},{\"index\":14,\"value\":8.9,\"date\":\"2023-07-24\"},{\"index\":15,\"value\":9.4,\"date\":\"2023-07-25\"},{\"index\":16,\"value\":6.5,\"date\":\"2023-07-26\"},{\"index\":17,\"value\":1.6,\"date\":\"2023-07-27\"},{\"index\":18,\"value\":-5.1,\"date\":\"2023-07-28\"},{\"index\":19,\"value\":-4.1,\"date\":\"2023-07-29\"},{\"index\":20,\"value\":-3.1,\"date\":\"2023-07-30\"},{\"index\":21,\"value\":-2.1,\"date\":\"2023-07-31\"},{\"index\":22,\"value\":-1.1,\"date\":\"2023-08-01\"},{\"index\":23,\"value\":-0.1,\"date\":\"2023-08-02\"},{\"index\":24,\"value\":0.9,\"date\":\"2023-08-03\"},{\"index\":25,\"value\":1.9,\"date\":\"2023-08-04\"},{\"index\":26,\"value\":2.9,\"date\":\"2023-08-05\"},{\"index\":27,\"value\":3.9,\"date\":\"2023-08-06\"},{\"index\":28,\"value\":4.9,\"date\":\"2023-08-07\"},{\"index\":29,\"value\":5.9,\"date\":\"2023-08-08\"},{\"index\":30,\"value\":6.9,\"date\":\"2023-08-09\"},{\"index\":31,\"value\":3.6,\"date\":\"2023-08-10\"},{\"index\":32,\"value\":8.9,\"date\":\"2023-08-11\"},{\"index\":33,\"value\":9.4,\"date\":\"2023-08-12\"},{\"index\":34,\"value\":0.9,\"date\":\"2023-08-12\"},{\"index\":35,\"value\":6.5,\"date\":\"2023-08-13\"},{\"index\":36,\"value\":1.6,\"date\":\"2023-08-14\"},{\"index\":37,\"value\":-5.1,\"date\":\"2023-08-15\"},{\"index\":38,\"value\":-4.1,\"date\":\"2023-08-16\"},{\"index\":39,\"value\":-3.1,\"date\":\"2023-08-17\"},{\"index\":40,\"value\":-2.1,\"date\":\"2023-08-18\"},{\"index\":41,\"value\":-1.1,\"date\":\"2023-08-19\"},{\"index\":42,\"value\":-0.1,\"date\":\"2023-08-20\"},{\"index\":43,\"value\":1.9,\"date\":\"2023-08-22\"},{\"index\":44,\"value\":2.9,\"date\":\"2023-08-23\"},{\"index\":45,\"value\":3.9,\"date\":\"2023-08-24\"},{\"index\":46,\"value\":4.9,\"date\":\"2023-08-25\"},{\"index\":47,\"value\":5.9,\"date\":\"2023-08-26\"},{\"index\":48,\"value\":6.9,\"date\":\"2023-08-27\"},{\"index\":49,\"value\":3.6,\"date\":\"2023-08-28\"},{\"index\":50,\"value\":8.9,\"date\":\"2023-08-29\"},{\"index\":51,\"value\":9.4,\"date\":\"2023-08-30\"},{\"index\":52,\"value\":6.5,\"date\":\"2023-08-31\"},{\"index\":53,\"value\":1.6,\"date\":\"2023-09-01\"},{\"index\":54,\"value\":-5.1,\"date\":\"2023-09-02\"},{\"index\":55,\"value\":-4.1,\"date\":\"2023-09-03\"},{\"index\":56,\"value\":-3.1,\"date\":\"2023-09-04\"},{\"index\":57,\"value\":-2.1,\"date\":\"2023-09-05\"},{\"index\":58,\"value\":-1.1,\"date\":\"2023-09-06\"},{\"index\":59,\"value\":-0.1,\"date\":\"2023-09-07\"},{\"index\":60,\"value\":0.9,\"date\":\"2023-09-08\"},{\"index\":61,\"value\":1.9,\"date\":\"2023-09-09\"},{\"index\":62,\"value\":2.9,\"date\":\"2023-09-10\"},{\"index\":63,\"value\":3.9,\"date\":\"2023-09-11\"},{\"index\":64,\"value\":4.9,\"date\":\"2023-09-12\"},{\"index\":65,\"value\":-4.1,\"date\":\"2023-09-12\"},{\"index\":66,\"value\":5.9,\"date\":\"2023-09-13\"},{\"index\":67,\"value\":6.9,\"date\":\"2023-09-14\"},{\"index\":68,\"value\":3.6,\"date\":\"2023-09-15\"},{\"index\":69,\"value\":8.9,\"date\":\"2023-09-16\"},{\"index\":70,\"value\":9.4,\"date\":\"2023-09-17\"},{\"index\":71,\"value\":6.5,\"date\":\"2023-09-18\"},{\"index\":72,\"value\":1.6,\"date\":\"2023-09-19\"},{\"index\":73,\"value\":-5.1,\"date\":\"2023-09-20\"},{\"index\":74,\"value\":-3.1,\"date\":\"2023-09-22\"},{\"index\":75,\"value\":-2.1,\"date\":\"2023-09-23\"},{\"index\":76,\"value\":-1.1,\"date\":\"2023-09-24\"},{\"index\":77,\"value\":-0.1,\"date\":\"2023-09-25\"},{\"index\":78,\"value\":0.9,\"date\":\"2023-09-26\"},{\"index\":79,\"value\":1.9,\"date\":\"2023-09-27\"},{\"index\":80,\"value\":2.9,\"date\":\"2023-09-28\"},{\"index\":81,\"value\":3.9,\"date\":\"2023-09-29\"},{\"index\":82,\"value\":4.9,\"date\":\"2023-09-30\"},{\"index\":83,\"value\":5.9,\"date\":\"2023-10-01\"},{\"index\":84,\"value\":6.9,\"date\":\"2023-10-02\"},{\"index\":85,\"value\":3.6,\"date\":\"2023-10-03\"},{\"index\":86,\"value\":8.9,\"date\":\"2023-10-04\"},{\"index\":87,\"value\":9.4,\"date\":\"2023-10-05\"},{\"index\":88,\"value\":6.5,\"date\":\"2023-10-06\"},{\"index\":89,\"value\":1.6,\"date\":\"2023-10-07\"},{\"index\":90,\"value\":-5.1,\"date\":\"2023-10-08\"},{\"index\":91,\"value\":-4.1,\"date\":\"2023-10-09\"},{\"index\":92,\"value\":-3.1,\"date\":\"2023-10-10\"},{\"index\":93,\"value\":-2.1,\"date\":\"2023-10-11\"},{\"index\":94,\"value\":3.6,\"date\":\"2023-10-12\"},{\"index\":95,\"value\":-1.1,\"date\":\"2023-10-12\"},{\"index\":96,\"value\":-0.1,\"date\":\"2023-10-13\"},{\"index\":97,\"value\":0.9,\"date\":\"2023-10-14\"},{\"index\":98,\"value\":1.9,\"date\":\"2023-10-15\"},{\"index\":99,\"value\":2.9,\"date\":\"2023-10-16\"},{\"index\":100,\"value\":3.9,\"date\":\"2023-10-17\"},{\"index\":101,\"value\":4.9,\"date\":\"2023-10-18\"},{\"index\":102,\"value\":5.9,\"date\":\"2023-10-19\"},{\"index\":103,\"value\":6.9,\"date\":\"2023-10-20\"},{\"index\":104,\"value\":8.9,\"date\":\"2023-10-22\"},{\"index\":105,\"value\":9.4,\"date\":\"2023-10-23\"},{\"index\":106,\"value\":6.5,\"date\":\"2023-10-24\"},{\"index\":107,\"value\":1.6,\"date\":\"2023-10-25\"},{\"index\":108,\"value\":-5.1,\"date\":\"2023-10-26\"},{\"index\":109,\"value\":-4.1,\"date\":\"2023-10-27\"},{\"index\":110,\"value\":-3.1,\"date\":\"2023-10-28\"},{\"index\":111,\"value\":-2.1,\"date\":\"2023-10-29\"},{\"index\":112,\"value\":-1.1,\"date\":\"2023-10-30\"},{\"index\":113,\"value\":-0.1,\"date\":\"2023-10-31\"},{\"index\":114,\"value\":0.9,\"date\":\"2023-11-01\"},{\"index\":115,\"value\":1.9,\"date\":\"2023-11-02\"},{\"index\":116,\"value\":2.9,\"date\":\"2023-11-03\"},{\"index\":117,\"value\":3.9,\"date\":\"2023-11-04\"},{\"index\":118,\"value\":4.9,\"date\":\"2023-11-05\"},{\"index\":119,\"value\":5.9,\"date\":\"2023-11-06\"},{\"index\":120,\"value\":6.9,\"date\":\"2023-11-07\"},{\"index\":121,\"value\":3.6,\"date\":\"2023-11-08\"},{\"index\":122,\"value\":8.9,\"date\":\"2023-11-09\"},{\"index\":123,\"value\":9.4,\"date\":\"2023-11-10\"},{\"index\":124,\"value\":6.5,\"date\":\"2023-11-11\"},{\"index\":125,\"value\":1.6,\"date\":\"2023-11-12\"},{\"index\":126,\"value\":-5.1,\"date\":\"2023-11-13\"},{\"index\":127,\"value\":-4.1,\"date\":\"2023-11-14\"},{\"index\":128,\"value\":-3.1,\"date\":\"2023-11-15\"},{\"index\":129,\"value\":-2.1,\"date\":\"2023-11-16\"},{\"index\":130,\"value\":-1.1,\"date\":\"2023-11-17\"},{\"index\":131,\"value\":-0.1,\"date\":\"2023-11-18\"},{\"index\":132,\"value\":0.9,\"date\":\"2023-11-19\"},{\"index\":133,\"value\":1.9,\"date\":\"2023-11-20\"},{\"index\":134,\"value\":2.9,\"date\":\"2023-11-21\"},{\"index\":135,\"value\":3.9,\"date\":\"2023-11-22\"},{\"index\":136,\"value\":4.9,\"date\":\"2023-11-23\"},{\"index\":137,\"value\":5.9,\"date\":\"2023-11-24\"},{\"index\":138,\"value\":6.9,\"date\":\"2023-11-25\"},{\"index\":139,\"value\":3.6,\"date\":\"2023-11-26\"},{\"index\":140,\"value\":8.9,\"date\":\"2023-11-27\"},{\"index\":141,\"value\":9.4,\"date\":\"2023-11-28\"},{\"index\":142,\"value\":6.5,\"date\":\"2023-11-29\"},{\"index\":143,\"value\":1.6,\"date\":\"2023-11-30\"},{\"index\":144,\"value\":-5.1,\"date\":\"2023-12-01\"},{\"index\":145,\"value\":-4.1,\"date\":\"2023-12-02\"},{\"index\":146,\"value\":-3.1,\"date\":\"2023-12-03\"},{\"index\":147,\"value\":-2.1,\"date\":\"2023-12-04\"},{\"index\":148,\"value\":-1.1,\"date\":\"2023-12-05\"},{\"index\":149,\"value\":-0.1,\"date\":\"2023-12-06\"},{\"index\":150,\"value\":0.9,\"date\":\"2023-12-07\"},{\"index\":151,\"value\":1.9,\"date\":\"2023-12-08\"},{\"index\":152,\"value\":2.9,\"date\":\"2023-12-09\"},{\"index\":153,\"value\":3.9,\"date\":\"2023-12-10\"},{\"index\":154,\"value\":4.9,\"date\":\"2023-12-11\"},{\"index\":155,\"value\":5.9,\"date\":\"2023-12-12\"},{\"index\":156,\"value\":6.9,\"date\":\"2023-12-13\"},{\"index\":157,\"value\":3.6,\"date\":\"2023-12-14\"},{\"index\":158,\"value\":8.9,\"date\":\"2023-12-15\"},{\"index\":159,\"value\":9.4,\"date\":\"2023-12-16\"},{\"index\":160,\"value\":6.5,\"date\":\"2023-12-17\"},{\"index\":161,\"value\":1.6,\"date\":\"2023-12-18\"},{\"index\":162,\"value\":-5.1,\"date\":\"2023-12-19\"},{\"index\":163,\"value\":-4.1,\"date\":\"2023-12-20\"},{\"index\":164,\"value\":-3.1,\"date\":\"2023-12-21\"},{\"index\":165,\"value\":-2.1,\"date\":\"2023-12-22\"},{\"index\":166,\"value\":-1.1,\"date\":\"2023-12-23\"},{\"index\":167,\"value\":-0.1,\"date\":\"2023-12-24\"},{\"index\":168,\"value\":0.9,\"date\":\"2023-12-25\"},{\"index\":169,\"value\":1.9,\"date\":\"2023-12-26\"},{\"index\":170,\"value\":2.9,\"date\":\"2023-12-27\"},{\"index\":171,\"value\":3.9,\"date\":\"2023-12-28\"},{\"index\":172,\"value\":4.9,\"date\":\"2023-12-29\"},{\"index\":173,\"value\":5.9,\"date\":\"2023-12-30\"},{\"index\":174,\"value\":6.9,\"date\":\"2023-12-31\"},{\"index\":175,\"value\":3.6,\"date\":\"2024-01-01\"},{\"index\":176,\"value\":8.9,\"date\":\"2024-01-02\"},{\"index\":177,\"value\":9.4,\"date\":\"2024-01-03\"},{\"index\":178,\"value\":6.5,\"date\":\"2024-01-04\"},{\"index\":179,\"value\":1.6,\"date\":\"2024-01-05\"},{\"index\":180,\"value\":-5.1,\"date\":\"2024-01-06\"},{\"index\":181,\"value\":-4.1,\"date\":\"2024-01-07\"},{\"index\":182,\"value\":-3.1,\"date\":\"2024-01-08\"},{\"index\":183,\"value\":-2.1,\"date\":\"2024-01-09\"},{\"index\":184,\"value\":-1.1,\"date\":\"2024-01-10\"},{\"index\":185,\"value\":-0.1,\"date\":\"2024-01-11\"},{\"index\":186,\"value\":0.9,\"date\":\"2024-01-12\"},{\"index\":187,\"value\":1.9,\"date\":\"2024-01-13\"},{\"index\":188,\"value\":2.9,\"date\":\"2024-01-14\"},{\"index\":189,\"value\":3.9,\"date\":\"2024-01-15\"},{\"index\":190,\"value\":4.9,\"date\":\"2024-01-16\"},{\"index\":191,\"value\":5.9,\"date\":\"2024-01-17\"},{\"index\":192,\"value\":6.9,\"date\":\"2024-01-18\"},{\"index\":193,\"value\":3.6,\"date\":\"2024-01-19\"},{\"index\":194,\"value\":8.9,\"date\":\"2024-01-20\"},{\"index\":195,\"value\":9.4,\"date\":\"2024-01-21\"},{\"index\":196,\"value\":6.5,\"date\":\"2024-01-22\"},{\"index\":197,\"value\":1.6,\"date\":\"2024-01-23\"},{\"index\":198,\"value\":3.6,\"date\":\"2024-01-24\"},{\"index\":199,\"value\":8.9,\"date\":\"2024-01-25\"},{\"index\":200,\"value\":9.4,\"date\":\"2024-01-26\"},{\"index\":201,\"value\":6.5,\"date\":\"2024-01-27\"},{\"index\":202,\"value\":1.6,\"date\":\"2024-01-28\"},{\"index\":203,\"value\":-5.1,\"date\":\"2024-01-29\"},{\"index\":204,\"value\":-4.1,\"date\":\"2024-01-30\"},{\"index\":205,\"value\":-3.1,\"date\":\"2024-01-31\"},{\"index\":206,\"value\":-2.1,\"date\":\"2024-02-01\"},{\"index\":207,\"value\":-1.1,\"date\":\"2024-02-02\"},{\"index\":208,\"value\":-0.1,\"date\":\"2024-02-03\"},{\"index\":209,\"value\":0.9,\"date\":\"2024-02-04\"},{\"index\":210,\"value\":1.9,\"date\":\"2024-02-05\"},{\"index\":211,\"value\":2.9,\"date\":\"2024-02-06\"},{\"index\":212,\"value\":3.9,\"date\":\"2024-02-07\"},{\"index\":213,\"value\":4.9,\"date\":\"2024-02-08\"},{\"index\":214,\"value\":5.9,\"date\":\"2024-02-09\"},{\"index\":215,\"value\":6.9,\"date\":\"2024-02-10\"},{\"index\":216,\"value\":3.6,\"date\":\"2024-02-11\"},{\"index\":217,\"value\":8.9,\"date\":\"2024-02-12\"},{\"index\":218,\"value\":9.4,\"date\":\"2024-02-13\"},{\"index\":219,\"value\":6.5,\"date\":\"2024-02-14\"},{\"index\":220,\"value\":1.6,\"date\":\"2024-02-15\"},{\"index\":221,\"value\":-5.1,\"date\":\"2024-02-16\"},{\"index\":222,\"value\":-4.1,\"date\":\"2024-02-17\"},{\"index\":223,\"value\":-3.1,\"date\":\"2024-02-18\"},{\"index\":224,\"value\":-2.1,\"date\":\"2024-02-19\"},{\"index\":225,\"value\":-1.1,\"date\":\"2024-02-20\"},{\"index\":226,\"value\":-0.1,\"date\":\"2024-02-21\"},{\"index\":227,\"value\":0.9,\"date\":\"2024-02-22\"},{\"index\":228,\"value\":1.9,\"date\":\"2024-02-23\"},{\"index\":229,\"value\":2.9,\"date\":\"2024-02-24\"},{\"index\":230,\"value\":3.9,\"date\":\"2024-02-25\"},{\"index\":231,\"value\":4.9,\"date\":\"2024-02-26\"},{\"index\":232,\"value\":5.9,\"date\":\"2024-02-27\"},{\"index\":233,\"value\":6.9,\"date\":\"2024-02-28\"},{\"index\":234,\"value\":3.6,\"date\":\"2024-02-29\"},{\"index\":235,\"value\":8.9,\"date\":\"2024-03-01\"},{\"index\":236,\"value\":9.4,\"date\":\"2024-03-02\"},{\"index\":237,\"value\":6.5,\"date\":\"2024-03-03\"},{\"index\":238,\"value\":1.6,\"date\":\"2024-03-04\"},{\"index\":239,\"value\":-5.1,\"date\":\"2024-03-05\"},{\"index\":240,\"value\":-4.1,\"date\":\"2024-03-06\"},{\"index\":241,\"value\":-3.1,\"date\":\"2024-03-07\"},{\"index\":242,\"value\":-2.1,\"date\":\"2024-03-08\"},{\"index\":243,\"value\":-1.1,\"date\":\"2024-03-09\"},{\"index\":244,\"value\":-0.1,\"date\":\"2024-03-10\"},{\"index\":245,\"value\":0.9,\"date\":\"2024-03-11\"},{\"index\":246,\"value\":1.9,\"date\":\"2024-03-12\"},{\"index\":247,\"value\":2.9,\"date\":\"2024-03-13\"},{\"index\":248,\"value\":3.9,\"date\":\"2024-03-14\"},{\"index\":249,\"value\":4.9,\"date\":\"2024-03-15\"},{\"index\":250,\"value\":5.9,\"date\":\"2024-03-16\"},{\"index\":251,\"value\":6.9,\"date\":\"2024-03-17\"},{\"index\":252,\"value\":3.6,\"date\":\"2024-03-18\"},{\"index\":253,\"value\":8.9,\"date\":\"2024-03-19\"},{\"index\":254,\"value\":9.4,\"date\":\"2024-03-20\"},{\"index\":255,\"value\":6.5,\"date\":\"2024-03-21\"},{\"index\":256,\"value\":1.6,\"date\":\"2024-03-22\"},{\"index\":257,\"value\":-5.1,\"date\":\"2024-03-23\"},{\"index\":258,\"value\":-4.1,\"date\":\"2024-03-24\"},{\"index\":259,\"value\":-3.1,\"date\":\"2024-03-25\"},{\"index\":260,\"value\":-2.1,\"date\":\"2024-03-26\"},{\"index\":261,\"value\":-1.1,\"date\":\"2024-03-27\"},{\"index\":262,\"value\":-0.1,\"date\":\"2024-03-28\"},{\"index\":263,\"value\":0.9,\"date\":\"2024-03-29\"},{\"index\":264,\"value\":1.9,\"date\":\"2024-03-30\"},{\"index\":265,\"value\":2.9,\"date\":\"2024-03-31\"},{\"index\":266,\"value\":3.9,\"date\":\"2024-04-01\"},{\"index\":267,\"value\":4.9,\"date\":\"2024-04-02\"},{\"index\":268,\"value\":5.9,\"date\":\"2024-04-03\"},{\"index\":269,\"value\":6.9,\"date\":\"2024-04-04\"},{\"index\":270,\"value\":3.6,\"date\":\"2024-04-05\"},{\"index\":271,\"value\":8.9,\"date\":\"2024-04-06\"},{\"index\":272,\"value\":9.4,\"date\":\"2024-04-07\"},{\"index\":273,\"value\":6.5,\"date\":\"2024-04-08\"},{\"index\":274,\"value\":1.6,\"date\":\"2024-04-09\"},{\"index\":275,\"value\":-5.1,\"date\":\"2024-04-10\"},{\"index\":276,\"value\":-4.1,\"date\":\"2024-04-11\"},{\"index\":277,\"value\":-3.1,\"date\":\"2024-04-12\"},{\"index\":278,\"value\":-2.1,\"date\":\"2024-04-13\"},{\"index\":279,\"value\":-1.1,\"date\":\"2024-04-14\"},{\"index\":280,\"value\":-0.1,\"date\":\"2024-04-15\"},{\"index\":281,\"value\":0.9,\"date\":\"2024-04-16\"},{\"index\":282,\"value\":1.9,\"date\":\"2024-04-17\"},{\"index\":283,\"value\":2.9,\"date\":\"2024-04-18\"},{\"index\":284,\"value\":3.9,\"date\":\"2024-04-19\"},{\"index\":285,\"value\":4.9,\"date\":\"2024-04-20\"},{\"index\":286,\"value\":5.9,\"date\":\"2024-04-21\"},{\"index\":287,\"value\":6.9,\"date\":\"2024-04-22\"},{\"index\":288,\"value\":3.6,\"date\":\"2024-04-23\"},{\"index\":289,\"value\":8.9,\"date\":\"2024-04-24\"},{\"index\":290,\"value\":9.4,\"date\":\"2024-04-25\"},{\"index\":291,\"value\":6.5,\"date\":\"2024-04-26\"},{\"index\":292,\"value\":1.6,\"date\":\"2024-04-27\"},{\"index\":293,\"value\":-5.1,\"date\":\"2024-04-28\"},{\"index\":294,\"value\":-4.1,\"date\":\"2024-04-29\"},{\"index\":295,\"value\":-3.1,\"date\":\"2024-04-30\"},{\"index\":296,\"value\":-2.1,\"date\":\"2024-05-01\"},{\"index\":297,\"value\":-1.1,\"date\":\"2024-05-02\"},{\"index\":298,\"value\":-0.1,\"date\":\"2024-05-03\"},{\"index\":299,\"value\":0.9,\"date\":\"2024-05-04\"},{\"index\":300,\"value\":1.9,\"date\":\"2024-05-05\"},{\"index\":301,\"value\":2.9,\"date\":\"2024-05-06\"},{\"index\":302,\"value\":3.9,\"date\":\"2024-05-07\"},{\"index\":303,\"value\":4.9,\"date\":\"2024-05-08\"},{\"index\":304,\"value\":5.9,\"date\":\"2024-05-09\"},{\"index\":305,\"value\":6.9,\"date\":\"2024-05-10\"},{\"index\":306,\"value\":3.6,\"date\":\"2024-05-11\"},{\"index\":307,\"value\":8.9,\"date\":\"2024-05-12\"},{\"index\":308,\"value\":9.4,\"date\":\"2024-05-13\"},{\"index\":309,\"value\":6.5,\"date\":\"2024-05-14\"},{\"index\":310,\"value\":1.6,\"date\":\"2024-05-15\"},{\"index\":311,\"value\":-5.1,\"date\":\"2024-05-16\"},{\"index\":312,\"value\":-4.1,\"date\":\"2024-05-17\"},{\"index\":313,\"value\":-3.1,\"date\":\"2024-05-18\"},{\"index\":314,\"value\":-2.1,\"date\":\"2024-05-19\"},{\"index\":315,\"value\":-1.1,\"date\":\"2024-05-20\"},{\"index\":316,\"value\":-0.1,\"date\":\"2024-05-21\"},{\"index\":317,\"value\":0.9,\"date\":\"2024-05-22\"},{\"index\":318,\"value\":1.9,\"date\":\"2024-05-23\"},{\"index\":319,\"value\":2.9,\"date\":\"2024-05-24\"},{\"index\":320,\"value\":3.9,\"date\":\"2024-05-25\"},{\"index\":321,\"value\":4.9,\"date\":\"2024-05-26\"},{\"index\":322,\"value\":5.9,\"date\":\"2024-05-27\"},{\"index\":323,\"value\":6.9,\"date\":\"2024-05-28\"},{\"index\":324,\"value\":3.6,\"date\":\"2024-05-29\"},{\"index\":325,\"value\":8.9,\"date\":\"2024-05-30\"},{\"index\":326,\"value\":9.4,\"date\":\"2024-05-31\"},{\"index\":327,\"value\":6.5,\"date\":\"2024-06-01\"},{\"index\":328,\"value\":1.6,\"date\":\"2024-06-02\"},{\"index\":329,\"value\":-5.1,\"date\":\"2024-06-03\"},{\"index\":330,\"value\":-4.1,\"date\":\"2024-06-04\"},{\"index\":331,\"value\":-3.1,\"date\":\"2024-06-05\"},{\"index\":332,\"value\":-2.1,\"date\":\"2024-06-06\"},{\"index\":333,\"value\":-1.1,\"date\":\"2024-06-07\"},{\"index\":334,\"value\":-0.1,\"date\":\"2024-06-08\"},{\"index\":335,\"value\":0.9,\"date\":\"2024-06-09\"},{\"index\":336,\"value\":1.9,\"date\":\"2024-06-10\"},{\"index\":337,\"value\":2.9,\"date\":\"2024-06-11\"},{\"index\":338,\"value\":3.9,\"date\":\"2024-06-12\"},{\"index\":339,\"value\":4.9,\"date\":\"2024-06-13\"},{\"index\":340,\"value\":5.9,\"date\":\"2024-06-14\"},{\"index\":341,\"value\":6.9,\"date\":\"2024-06-15\"},{\"index\":342,\"value\":3.6,\"date\":\"2024-06-16\"},{\"index\":343,\"value\":8.9,\"date\":\"2024-06-17\"},{\"index\":344,\"value\":9.4,\"date\":\"2024-06-18\"},{\"index\":345,\"value\":6.5,\"date\":\"2024-06-19\"},{\"index\":346,\"value\":1.6,\"date\":\"2024-06-20\"},{\"index\":347,\"value\":-5.1,\"date\":\"2024-06-21\"},{\"index\":348,\"value\":-4.1,\"date\":\"2024-06-22\"},{\"index\":349,\"value\":-3.1,\"date\":\"2024-06-23\"},{\"index\":350,\"value\":-2.1,\"date\":\"2024-06-24\"},{\"index\":351,\"value\":-1.1,\"date\":\"2024-06-25\"},{\"index\":352,\"value\":-0.1,\"date\":\"2024-06-26\"},{\"index\":353,\"value\":0.9,\"date\":\"2024-06-27\"},{\"index\":354,\"value\":1.9,\"date\":\"2024-06-28\"},{\"index\":355,\"value\":2.9,\"date\":\"2024-06-29\"},{\"index\":356,\"value\":3.9,\"date\":\"2024-06-30\"},{\"index\":357,\"value\":4.9,\"date\":\"2024-07-01\"},{\"index\":358,\"value\":5.9,\"date\":\"2024-07-02\"},{\"index\":359,\"value\":6.9,\"date\":\"2024-07-03\"},{\"index\":360,\"value\":3.6,\"date\":\"2024-07-04\"},{\"index\":361,\"value\":8.9,\"date\":\"2024-07-05\"},{\"index\":362,\"value\":9.4,\"date\":\"2024-07-06\"},{\"index\":363,\"value\":6.5,\"date\":\"2024-07-07\"},{\"index\":364,\"value\":1.6,\"date\":\"2024-07-08\"}]","id":"2023-07-10","start_date":"2023-07-10"}]}]}

# pre-process data
product_id = 'RYJJ-06-0002'
data_wrapper = eval(response['data'][0][product_id][0]['data'])
data = [item['value'] for item in data_wrapper]
data = [MapRange(v, -20, 20, 0, 1) for v in data]
print(data)

# determine the path
refresh_date = datetime.now().strftime("%Y%m%d_%H%M%S")
dst_image_folder = '/Users/ahu01/Desktop/GenerateLine'
if not os.path.exists(dst_image_folder):
    os.makedirs(dst_image_folder)
dst_image_path = dst_image_folder + '/' +  product_id + '_' + refresh_date + '.png'

# genertate image
generate_line_image_by_data(data, dst_image_path, point_count=52, image_size=(512, 256), line_color=(0, 0, 0, 255), line_width = 1, image_margin=(0, 0))